In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from generation import MarkedIntensityHomogenuosPoisson, generate_samples_marked


In [27]:
class PointProcessDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = [
            torch.tensor([[event[0], event[1]] for event in seq], dtype=torch.float32)
            for seq in sequences
        ]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx]
def collate_fn(batch):
    return batch
DIM_SIZE = 7
BATCH_SIZE=256
mi = MarkedIntensityHomogenuosPoisson(DIM_SIZE)
for u in range(DIM_SIZE):
    mi.initialize(1.0, u)
simulated_sequences = generate_samples_marked(mi, 20.0, 10000)
dataset = PointProcessDataset(simulated_sequences)
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_fn
)


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleGRU(nn.Module):
    def __init__(self, num_classes, hidden_size_event, hidden_size_time, reg=0.1):
        super(SimpleGRU, self).__init__()
        self.num_classes = num_classes
        self.reg = reg
        self.event_embedding = nn.Embedding(num_classes, num_classes)
        self.event_gru = nn.GRU(num_classes, hidden_size_event, batch_first=True)
        self.time_gru = nn.GRU(1, hidden_size_time, batch_first=True)
        combined_size = hidden_size_event + hidden_size_time
        self.time_output = nn.Linear(combined_size, 1)
        self.mark_output = nn.Linear(combined_size, num_classes)

    def forward(self, event_sequence):
        marks = event_sequence[..., 0].long()
        times = event_sequence[..., 1].unsqueeze(-1)
        mark_embedded = self.event_embedding(marks)
        event_output, _ = self.event_gru(mark_embedded)
        time_output, _ = self.time_gru(times)
        combined_output = torch.cat([event_output, time_output], dim=-1)
        time_pred = self.time_output(combined_output)
        mark_logits = self.mark_output(combined_output)
        return time_pred, mark_logits
    def compute_loss(self, time_pred, mark_logits, targets):
        true_times = targets[..., 1].unsqueeze(-1)
        true_marks = targets[..., 0].long()
        time_loss = F.mse_loss(time_pred, true_times)
        mark_logits_flat = mark_logits.view(-1, self.num_classes)
        true_marks_flat = true_marks.view(-1)
        mark_loss = F.cross_entropy(mark_logits_flat, true_marks_flat)
        total_loss = mark_loss + self.reg * time_loss
        return total_loss, mark_loss, time_loss


In [29]:
from tqdm import tqdm  # 导入 tqdm 函数

def train_model(model, train_loader, num_epochs=10, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    """
    训练模型的函数
    
    Args:
        model: SimpleGRU模型实例
        train_loader: 训练数据的DataLoader
        num_epochs: 训练轮数
        learning_rate: 学习率
        device: 训练设备（'cuda'或'cpu'）
    """
    print(f"Training on {device}")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        train_total_loss = 0
        train_mark_loss_sum = 0
        train_time_loss_sum = 0
        num_batches = 0
        
        epoch_start_time = time.time()
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch in train_bar:
            optimizer.zero_grad()
            batch_loss = 0
            batch_mark_loss = 0
            batch_time_loss = 0
            
            for sequence in batch:
                sequence = sequence.to(device)
                sequence = sequence.unsqueeze(0)
                time_pred, mark_logits = model(sequence)
                loss, mark_loss, time_loss = model.compute_loss(time_pred, mark_logits, sequence)
                batch_loss += loss
                batch_mark_loss += mark_loss
                batch_time_loss += time_loss
            batch_size = len(batch)
            batch_loss /= batch_size
            batch_mark_loss /= batch_size
            batch_time_loss /= batch_size
            batch_loss.backward()
            optimizer.step()
            train_total_loss += batch_loss.item()
            train_mark_loss_sum += batch_mark_loss.item()
            train_time_loss_sum += batch_time_loss.item()
            num_batches += 1
            
            train_bar.set_postfix({
                'loss': f'{batch_loss.item():.4f}',
                'mark_loss': f'{batch_mark_loss.item():.4f}',
                'time_loss': f'{batch_time_loss.item():.4f}'
            })
        
        avg_train_loss = train_total_loss / num_batches
        avg_train_mark_loss = train_mark_loss_sum / num_batches
        avg_train_time_loss = train_time_loss_sum / num_batches
        
        epoch_time = time.time() - epoch_start_time
        print(f'\nEpoch {epoch+1}/{num_epochs} - {epoch_time:.2f}s')
        print(f'Train Loss: {avg_train_loss:.4f} (Mark: {avg_train_mark_loss:.4f}, Time: {avg_train_time_loss:.4f})')
        print('-' * 80)

# 使用示例：
if __name__ == "__main__":
    # 创建模型
    model = SimpleGRU(
        num_classes=DIM_SIZE,
        hidden_size_event=16,
        hidden_size_time=32
    )
    
    train_model(
        model=model,
        train_loader=dataloader,
        num_epochs=100,
        learning_rate=0.001
    )


Training on cuda


Epoch 1/100: 100%|██████████| 40/40 [00:15<00:00,  2.53it/s, loss=11.5613, mark_loss=1.8686, time_loss=96.9274] 



Epoch 1/100 - 15.80s
Train Loss: 13.6339 (Mark: 1.9144, Time: 117.1947)
--------------------------------------------------------------------------------


Epoch 2/100: 100%|██████████| 40/40 [00:15<00:00,  2.56it/s, loss=7.9704, mark_loss=1.7912, time_loss=61.7925] 



Epoch 2/100 - 15.61s
Train Loss: 9.4751 (Mark: 1.8251, Time: 76.4998)
--------------------------------------------------------------------------------


...


Epoch 47/100 - 15.84s
Train Loss: 0.0215 (Mark: 0.0183, Time: 0.0319)
--------------------------------------------------------------------------------


Epoch 48/100: 100%|██████████| 40/40 [00:16<00:00,  2.48it/s, loss=0.0200, mark_loss=0.0174, time_loss=0.0261]



Epoch 48/100 - 16.14s
Train Loss: 0.0206 (Mark: 0.0176, Time: 0.0296)
--------------------------------------------------------------------------------


Epoch 49/100:  15%|█▌        | 6/40 [00:02<00:16,  2.11it/s, loss=0.0199, mark_loss=0.0172, time_loss=0.0276]


KeyboardInterrupt: 

In [32]:
class PointProcessDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = []
        for seq in sequences:
            if len(seq) >= 2:  # 确保序列至少有2个事件
                # 将序列转换为tensor
                seq_tensor = torch.tensor([[event[0], event[1]] for event in seq], dtype=torch.float32)
                # 分离输入序列(x)和目标值(y)
                x = seq_tensor[:-1]  # 除最后一个外的所有事件
                y = seq_tensor[-1]   # 最后一个事件
                self.sequences.append((x, y))
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx]

def collate_fn(batch):
    # batch中的每个元素都是(x, y)对
    return batch

# 生成数据
DIM_SIZE = 7
BATCH_SIZE = 256
mi = MarkedIntensityHomogenuosPoisson(DIM_SIZE)
for u in range(DIM_SIZE):
    mi.initialize(1.0, u)
simulated_sequences = generate_samples_marked(mi, 20.0, 10000)

# 创建数据集和数据加载器
dataset = PointProcessDataset(simulated_sequences)
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_fn
)

import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleGRU(nn.Module):
    def __init__(self, num_classes, hidden_size_event, hidden_size_time, reg=0.1):
        super(SimpleGRU, self).__init__()
        self.num_classes = num_classes
        self.reg = reg
        self.event_embedding = nn.Embedding(num_classes, num_classes)
        self.event_gru = nn.GRU(num_classes, hidden_size_event, batch_first=True)
        self.time_gru = nn.GRU(1, hidden_size_time, batch_first=True)
        combined_size = hidden_size_event + hidden_size_time
        self.time_output = nn.Linear(combined_size, 1)
        self.mark_output = nn.Linear(combined_size, num_classes)

    def forward(self, event_sequence):
        marks = event_sequence[..., 0].long()
        times = event_sequence[..., 1].unsqueeze(-1)
        
        # 处理所有时间步
        mark_embedded = self.event_embedding(marks)
        event_output, _ = self.event_gru(mark_embedded)
        time_output, _ = self.time_gru(times)
        
        # 只取最后一个时间步的输出
        combined_output = torch.cat([
            event_output[:, -1:, :],  # 只取最后一步
            time_output[:, -1:, :]    # 只取最后一步
        ], dim=-1)
        
        time_pred = self.time_output(combined_output)
        mark_logits = self.mark_output(combined_output)
        return time_pred, mark_logits

    def compute_loss(self, time_pred, mark_logits, targets):
        # 只取最后一步的真实值
        true_times = targets[..., -1:, 1].unsqueeze(-1)  # 最后一步的时间
        true_marks = targets[..., -1, 0].long()          # 最后一步的标记
        
        # 计算损失
        time_loss = F.mse_loss(time_pred, true_times)
        mark_logits_flat = mark_logits.view(-1, self.num_classes)
        true_marks_flat = true_marks.view(-1)
        mark_loss = F.cross_entropy(mark_logits_flat, true_marks_flat)
        
        total_loss = mark_loss + self.reg * time_loss
        return total_loss, mark_loss, time_loss
def train_model(model, train_loader, num_epochs=10, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    print(f"Training on {device}")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        train_total_loss = 0
        train_mark_loss_sum = 0
        train_time_loss_sum = 0
        num_batches = 0
        
        epoch_start_time = time.time()
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch in train_bar:
            optimizer.zero_grad()
            batch_loss = 0
            batch_mark_loss = 0
            batch_time_loss = 0
            
            for x, y in batch:
                x = x.to(device)
                y = y.to(device)
                x = x.unsqueeze(0)  # 添加batch维度
                y = y.unsqueeze(0)  # 添加batch维度
                
                # 前向传播
                time_pred, mark_logits = model(x)
                
                # 计算损失
                loss, mark_loss, time_loss = model.compute_loss(time_pred, mark_logits, y)
                
                batch_loss += loss
                batch_mark_loss += mark_loss
                batch_time_loss += time_loss
            
            batch_size = len(batch)
            batch_loss /= batch_size
            batch_mark_loss /= batch_size
            batch_time_loss /= batch_size
            
            batch_loss.backward()
            optimizer.step()
            
            train_total_loss += batch_loss.item()
            train_mark_loss_sum += batch_mark_loss.item()
            train_time_loss_sum += batch_time_loss.item()
            num_batches += 1
            
            train_bar.set_postfix({
                'loss': f'{batch_loss.item():.4f}',
                'mark_loss': f'{batch_mark_loss.item():.4f}',
                'time_loss': f'{batch_time_loss.item():.4f}'
            })
        
        avg_train_loss = train_total_loss / num_batches
        avg_train_mark_loss = train_mark_loss_sum / num_batches
        avg_train_time_loss = train_time_loss_sum / num_batches
        
        epoch_time = time.time() - epoch_start_time
        print(f'\nEpoch {epoch+1}/{num_epochs} - {epoch_time:.2f}s')
        print(f'Train Loss: {avg_train_loss:.4f} (Mark: {avg_train_mark_loss:.4f}, Time: {avg_train_time_loss:.4f})')
        print('-' * 80)

In [33]:
# 创建模型
model = SimpleGRU(
    num_classes=DIM_SIZE,
    hidden_size_event=32,
    hidden_size_time=32,
    reg=0.1
)

# 设置训练参数
num_epochs = 20
learning_rate = 0.001

# 训练模型
train_model(
    model=model,
    train_loader=dataloader,
    num_epochs=num_epochs,
    learning_rate=learning_rate
)

# # 可选：保存模型
# torch.save(model.state_dict(), 'simple_gru_model.pth')

# # 可选：预测示例
# model.eval()  # 设置为评估模式
# with torch.no_grad():
#     # 获取一个样本
#     x_sample, y_sample = next(iter(dataloader))[0]
#     x_sample = x_sample.unsqueeze(0)  # 添加batch维度
    
#     # 预测
#     time_pred, mark_logits = model(x_sample)
    
#     # 获取预测的标记类别
#     predicted_mark = torch.argmax(mark_logits, dim=-1)
    
#     print("\n预测示例:")
#     print(f"实际值 - 标记: {y_sample[0].item()}, 时间: {y_sample[1].item():.4f}")
#     print(f"预测值 - 标记: {predicted_mark.item()}, 时间: {time_pred.item():.4f}")

Training on cuda


Epoch 1/20:   0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_897/3030971952.py:81: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  time_loss = F.mse_loss(time_pred, true_times)
Epoch 1/20: 100%|██████████| 40/40 [00:17<00:00,  2.32it/s, loss=31.2713, mark_loss=1.9048, time_loss=293.6648]



Epoch 1/20 - 17.27s
Train Loss: 38.3748 (Mark: 1.9589, Time: 364.1587)
--------------------------------------------------------------------------------


Epoch 2/20: 100%|██████████| 40/40 [00:16<00:00,  2.39it/s, loss=18.8100, mark_loss=1.9452, time_loss=168.6483]



Epoch 2/20 - 16.75s
Train Loss: 24.0759 (Mark: 1.9490, Time: 221.2686)
--------------------------------------------------------------------------------


...


Epoch 18/20 - 17.24s
Train Loss: 1.9509 (Mark: 1.9487, Time: 0.0222)
--------------------------------------------------------------------------------


Epoch 19/20: 100%|██████████| 40/40 [00:16<00:00,  2.37it/s, loss=1.9407, mark_loss=1.9393, time_loss=0.0141]



Epoch 19/20 - 16.87s
Train Loss: 1.9505 (Mark: 1.9484, Time: 0.0214)
--------------------------------------------------------------------------------


Epoch 20/20: 100%|██████████| 40/40 [00:16<00:00,  2.40it/s, loss=1.9477, mark_loss=1.9445, time_loss=0.0323]


Epoch 20/20 - 16.69s
Train Loss: 1.9512 (Mark: 1.9490, Time: 0.0216)
--------------------------------------------------------------------------------


In [38]:

class PointProcessDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = []
        for seq in sequences:
            if len(seq) >= 2:  # 确保序列至少有2个事件
                # 将序列转换为tensor
                seq_tensor = torch.tensor([[event[0], event[1]] for event in seq], dtype=torch.float32)
                # 分离输入序列(x)和目标值(y)
                x = seq_tensor[:-1]  # 除最后一个外的所有事件
                y = seq_tensor[-1]   # 最后一个事件
                self.sequences.append((x, y))
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx]

def collate_fn(batch):
    return batch

class SimpleGRU(nn.Module):
    def __init__(self, num_classes, hidden_size_event, hidden_size_time, reg=0.1):
        super(SimpleGRU, self).__init__()
        self.num_classes = num_classes
        self.reg = reg
        self.event_embedding = nn.Embedding(num_classes, num_classes)
        self.event_gru = nn.GRU(num_classes, hidden_size_event, batch_first=True)
        self.time_gru = nn.GRU(1, hidden_size_time, batch_first=True)
        combined_size = hidden_size_event + hidden_size_time
        self.time_output = nn.Linear(combined_size, 1)
        self.mark_output = nn.Linear(combined_size, num_classes)

    def forward(self, event_sequence):
        marks = event_sequence[..., 0].long()
        times = event_sequence[..., 1].unsqueeze(-1)
        
        mark_embedded = self.event_embedding(marks)
        event_output, _ = self.event_gru(mark_embedded)
        time_output, _ = self.time_gru(times)
        
        combined_output = torch.cat([
            event_output[:, -1, :],
            time_output[:, -1, :]
        ], dim=-1)
        
        time_pred = self.time_output(combined_output).unsqueeze(1)
        mark_logits = self.mark_output(combined_output).unsqueeze(1)
        return time_pred, mark_logits

    def compute_loss(self, time_pred, mark_logits, targets):
        true_times = targets[..., 1].unsqueeze(-1)
        true_marks = targets[..., 0].long()
        
        time_loss = F.mse_loss(time_pred, true_times)
        mark_logits_flat = mark_logits.squeeze(1)  # 移除多余的维度
        mark_loss = F.cross_entropy(mark_logits_flat, true_marks)
        
        total_loss = mark_loss + self.reg * time_loss
        return total_loss, mark_loss, time_loss

def train_model(model, train_loader, num_epochs=10, learning_rate=0.001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    print(f"Training on {device}")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        train_total_loss = 0
        train_mark_loss_sum = 0
        train_time_loss_sum = 0
        num_batches = 0
        
        epoch_start_time = time.time()
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch in train_bar:
            optimizer.zero_grad()
            batch_loss = 0
            batch_mark_loss = 0
            batch_time_loss = 0
            
            for x, y in batch:
                x = x.to(device)
                y = y.to(device)
                x = x.unsqueeze(0)  # 添加batch维度
                y = y.unsqueeze(0)  # 添加batch维度
                
                # 前向传播
                time_pred, mark_logits = model(x)
                
                # 计算损失
                loss, mark_loss, time_loss = model.compute_loss(time_pred, mark_logits, y)
                
                batch_loss += loss
                batch_mark_loss += mark_loss
                batch_time_loss += time_loss
            
            batch_size = len(batch)
            batch_loss /= batch_size
            batch_mark_loss /= batch_size
            batch_time_loss /= batch_size
            
            batch_loss.backward()
            optimizer.step()
            
            train_total_loss += batch_loss.item()
            train_mark_loss_sum += batch_mark_loss.item()
            train_time_loss_sum += batch_time_loss.item()
            num_batches += 1
            
            train_bar.set_postfix({
                'loss': f'{batch_loss.item():.4f}',
                'mark_loss': f'{batch_mark_loss.item():.4f}',
                'time_loss': f'{batch_time_loss.item():.4f}'
            })
        
        avg_train_loss = train_total_loss / num_batches
        avg_train_mark_loss = train_mark_loss_sum / num_batches
        avg_train_time_loss = train_time_loss_sum / num_batches
        
        epoch_time = time.time() - epoch_start_time
        print(f'\nEpoch {epoch+1}/{num_epochs} - {epoch_time:.2f}s')
        print(f'Train Loss: {avg_train_loss:.4f} (Mark: {avg_train_mark_loss:.4f}, Time: {avg_train_time_loss:.4f})')
        
        # 每5个epoch打印预测示例
        if (epoch + 1) % 5 == 0:
            model.eval()
            with torch.no_grad():
                x_sample, y_sample = next(iter(train_loader))[0]
                x_sample = x_sample.to(device)
                x_sample = x_sample.unsqueeze(0)
                
                time_pred, mark_logits = model(x_sample)
                predicted_mark = torch.argmax(mark_logits.squeeze(), dim=-1)
                
                print("\n预测示例:")
                print(f"实际标记: {y_sample[0].item()}")
                print(f"预测标记: {predicted_mark.item()}")
                print(f"标记概率分布: {torch.softmax(mark_logits.squeeze(), dim=-1)}")
            model.train()
        
        print('-' * 80)

# 生成数据
DIM_SIZE = 1
BATCH_SIZE = 256
mi = MarkedIntensityHomogenuosPoisson(DIM_SIZE)
for u in range(DIM_SIZE):
    mi.initialize(1.0, u)
simulated_sequences = generate_samples_marked(mi, 100.0, 10000)

# 创建数据集和数据加载器
dataset = PointProcessDataset(simulated_sequences)

# 检查数据分布
mark_counts = torch.zeros(DIM_SIZE)
for x, y in dataset:
    mark_counts[int(y[0])] += 1

print("\n标记分布:")
for i in range(DIM_SIZE):
    print(f"标记 {i}: {mark_counts[i]} 个样本 ({mark_counts[i]/len(dataset)*100:.2f}%)")

# 创建数据加载器
dataloader = DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_fn
)

# 创建和训练模型
model = SimpleGRU(
    num_classes=DIM_SIZE,
    hidden_size_event=64,
    hidden_size_time=64,
    reg=0.1
)

# 训练模型
train_model(
    model=model,
    train_loader=dataloader,
    num_epochs=50,
    learning_rate=0.001
)

# 保存模型
torch.save(model.state_dict(), 'simple_gru_model.pth')

# 测试预测
model.eval()
with torch.no_grad():
    x_sample, y_sample = next(iter(dataloader))[0]
    x_sample = x_sample.unsqueeze(0)
    
    time_pred, mark_logits = model(x_sample)
    predicted_mark = torch.argmax(mark_logits.squeeze(), dim=-1)
    
    print("\n最终预测示例:")
    print(f"实际值 - 标记: {y_sample[0].item()}, 时间: {y_sample[1].item():.4f}")
    print(f"预测值 - 标记: {predicted_mark.item()}, 时间: {time_pred.squeeze().item():.4f}")


标记分布:
标记 0: 10000.0 个样本 (100.00%)
Training on cuda


Epoch 1/50:   0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_897/82287682.py:55: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  time_loss = F.mse_loss(time_pred, true_times)
Epoch 1/50: 100%|██████████| 40/40 [00:17<00:00,  2.34it/s, loss=854.1581, mark_loss=0.0000, time_loss=8541.5811]



Epoch 1/50 - 17.07s
Train Loss: 930.8562 (Mark: 0.0000, Time: 9308.5615)
--------------------------------------------------------------------------------


Epoch 2/50: 100%|██████████| 40/40 [00:16<00:00,  2.39it/s, loss=737.1065, mark_loss=0.0000, time_loss=7371.0649]



Epoch 2/50 - 16.70s
Train Loss: 794.5440 (Mark: 0.0000, Time: 7945.4399)
--------------------------------------------------------------------------------


...

Epoch 49/50: 100%|██████████| 40/40 [00:16<00:00,  2.38it/s, loss=0.1994, mark_loss=0.0000, time_loss=1.9941]



Epoch 49/50 - 16.81s
Train Loss: 0.1012 (Mark: 0.0000, Time: 1.0116)
--------------------------------------------------------------------------------


Epoch 50/50: 100%|██████████| 40/40 [00:16<00:00,  2.39it/s, loss=0.0950, mark_loss=0.0000, time_loss=0.9498]


Epoch 50/50 - 16.73s
Train Loss: 0.0976 (Mark: 0.0000, Time: 0.9761)

预测示例:
实际标记: 0.0
预测标记: 0
标记概率分布: 1.0
--------------------------------------------------------------------------------


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [4]:
import pandas as pd
from model import PointProcessDataset
df_test=pd.read_csv("/mnt/lia/scratch/wenqliu/ee556-2/master-project/point_process/mu_0.9_alpha_0.8_beta_1.0_T_1000_cluster.csv")
test_dataset = PointProcessDataset(T_test, time_step)

ModuleNotFoundError: No module named 'torch'

In [6]:
pip install torch

     |████████████████████████████████| 797.1 MB 21 kB/s s eta 0:00:01     |██████▊                         | 166.2 MB 41.6 MB/s eta 0:00:16     |███████▋                        | 188.3 MB 18.9 MB/s eta 0:00:33     |████████                        | 201.9 MB 33.9 MB/s eta 0:00:18     |█████████                       | 221.6 MB 33.9 MB/s eta 0:00:17     |█████████████▏                  | 327.0 MB 26.9 MB/s eta 0:00:18     |████████████████▌               | 411.9 MB 35.1 MB/s eta 0:00:11     |██████████████████▎             | 455.6 MB 39.1 MB/s eta 0:00:09     |███████████████████████▎        | 579.9 MB 70.4 MB/s eta 0:00:04     |███████████████████████▊        | 591.7 MB 70.4 MB/s eta 0:00:03     |████████████████████████        | 598.0 MB 17.8 MB/s eta 0:00:12     |█████████████████████████▉      | 642.9 MB 53.6 MB/s eta 0:00:03
     |████████████████████████████████| 14.1 MB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 27.8 MB/s eta 0:00:01
     |███████████████